### Задание 1-2

Нормализация (1НФ → 3НФ)
Перевод в 1НФ (удаляем повторяющиеся записи, создаём уникальные идентификаторы)

Данные уже находятся в 1НФ.
Перевод во 2НФ (удаляем частичные зависимости от составных ключей)

Разделение Products позволяет избежать дублирования информации о товарах в Transactions.
Перевод в 3НФ (удаляем зависимости)

Вынесение Addresses из Customers избавляет от дублирования адресов.

![Схема базы данных](db_schema.png)

### Задание 3

Код создания таблиц:
```
CREATE TABLE IF NOT EXISTS Customers (
    customer_id INT PRIMARY KEY,
    first_name VARCHAR(100),
    last_name VARCHAR(100),
    gender VARCHAR(10),
    DOB DATE,
    job_title VARCHAR(255),
    job_industry_category VARCHAR(255),
    wealth_segment VARCHAR(50),
    deceased_indicator CHAR(1),
    owns_car VARCHAR(10),
    property_valuation INT
);

CREATE TABLE IF NOT EXISTS Addresses (
    address_id SERIAL PRIMARY KEY,
    customer_id INT REFERENCES Customers(customer_id),
    address VARCHAR(255),
    postcode VARCHAR(10),
    state VARCHAR(50),
    country VARCHAR(50)
);

CREATE TABLE IF NOT EXISTS Products (
    product_id INT PRIMARY KEY,
    brand VARCHAR(100),
    product_line VARCHAR(100),
    product_class VARCHAR(50),
    product_size VARCHAR(50),
    list_price DECIMAL(10,2),
    standard_cost DECIMAL(10,2)
);

CREATE TABLE IF NOT EXISTS Transactions (
    transaction_id INT PRIMARY KEY,
    customer_id INT REFERENCES Customers(customer_id),
    product_id INT REFERENCES Products(product_id),
    transaction_date DATE,
    online_order BOOLEAN,
    order_status VARCHAR(50)
);
```


#### Задание 4

In [12]:
import pandas as pd
import psycopg2
from psycopg2.extras import execute_values

conn = psycopg2.connect(
    dbname="shod_1",
    user="dmp",
    password="dmp",
    host="localhost",
    port="5434"
)
cursor = conn.cursor()

file_path = "customer_and_transaction.xlsx"
sheets = pd.read_excel(file_path, sheet_name=None)

# Приводим названия колонок к нижнему регистру и убираем пробелы
df_customers = sheets["customer"]
df_customers.columns = df_customers.columns.str.strip().str.lower()

df_transactions = sheets["transaction"]
df_transactions.columns = df_transactions.columns.str.strip().str.lower()


# Преобразование типов
df_customers["dob"] = pd.to_datetime(df_customers["dob"], errors='coerce').dt.date
df_customers = df_customers.where(pd.notna(df_customers), None)

df_transactions["transaction_date"] = pd.to_datetime(df_transactions["transaction_date"], errors='coerce').dt.date
df_transactions = df_transactions.where(pd.notna(df_transactions), None)

# 1. Customers
df_customers_cleaned = df_customers[[
    "customer_id", "first_name", "last_name", "gender", "dob", "job_title",
    "job_industry_category", "wealth_segment", "deceased_indicator", "owns_car", "property_valuation"
]].drop_duplicates()

# 2. Addresses
df_addresses = df_customers[["customer_id", "address", "postcode", "state", "country"]].drop_duplicates()

# 3. Products
df_products = df_transactions[[
    "product_id", "brand", "product_line", "product_class", "product_size", "list_price", "standard_cost"
]].drop_duplicates()

# 4. Transactions
df_transactions_cleaned = df_transactions[[
    "transaction_id", "customer_id", "product_id", "transaction_date", "online_order", "order_status"
]]

# Фильтрация транзакций по существующим customer_id
df_transactions_cleaned = df_transactions_cleaned[
    df_transactions_cleaned["customer_id"].isin(df_customers_cleaned["customer_id"])
]


def insert_data(table_name, dataframe):
    
    columns = ', '.join(dataframe.columns)
    sql = f"INSERT INTO {table_name} ({columns}) VALUES %s ON CONFLICT DO NOTHING;"
    
    execute_values(cursor, sql, dataframe.values.tolist())
    conn.commit()
    print(f"Данные загружены в {table_name}")

insert_data("customers", df_customers_cleaned)
insert_data("addresses", df_addresses)
insert_data("products", df_products)
insert_data("transactions", df_transactions_cleaned)

cursor.close()
conn.close()



Данные загружены в customers
Данные загружены в addresses
Данные загружены в products
Данные загружены в transactions
